# Tutoriel d'introduction à Tensorflow Quantum
#### Auteurs: Ovigne Adrien, Verdier Fabien, Skaf Paul et Klaas Guillaume

## Sommaire : 

0. Introduction
1. Présentation de Cirq
2. Introduction de Tensorflow Quantum et exemple d'application
3. Conclusion
4. Bibliographie et Sources

## 0. Introduction:

Ce notebook vise à introduire les principales fonctionnalités de Tensorflow Quantum (TFQ par la suite). Ainsi nous allons commencer par Cirq, la librairie utilisée par TFQ pour produire des réseaux de neurones quantiques (QNN par la suite). Puis nous expliquerons les bases de TFQ.

Maintenant que le but de ce tutoriel a été établi, passons à une brève introduction des librairies et framework utilisées, Cirq est une librairie écrite par Google et actuellement (V0.8) en Alpha. TFQ est un framework créé par Google pour faire de l'apprentissage machine quantique (QML par la suite) inspiré par Tensorflow, le framework d'apprentissage machine créé par Google.


## 1. Présentation de Cirq:

[Cirq](https://cirq.readthedocs.io/en/stable/) est une librairie créée par Google pour l'informatique quantique, elle permet de simuler l'exécution de circuits quantiques ou de les exécuter sur des processeurs quantiques.

 La première étape est la création de qubits, Cirq possède trois méthodes pour cela: 

 * `cirq.NamedQubit` 
  
 * `cirq.LineQubit`
  
 * `cirq.GridQubit`
   
 Leur usage est décrit ci-dessous:
 

In [ ]:
import cirq


# Pour créer des Qubits nommés, ici l'objet q0 est nommé qubit 0.
q0 = cirq.NamedQubit('qubit 0')
q1 = cirq.NamedQubit('qubit 1')

# Pour créer des Qubits labelisés par leur index dans une file: ici le qubit q3 est généré avec le label '3'
q3 = cirq.LineQubit(3)

"""
On peut également utiliser cirq.LineQubit pour créer plusieurs Qubits à la fois, dans ce cas,
q0 porte le label 0, q1 le label 1 et q2 le label 2, 
"""
q0, q1, q2 = cirq.LineQubit.range(3)

# cirq.GridQubit permet de créer des Qubits sur 2 axes
q4_5 = cirq.GridQubit(4,5)
"""
Cette méthode peut également créer une grille de Qubits 
la ligne ci-dessous va créer une grille de 16 Qubits indexés de (0,0) à (3,3)
"""
qubits = cirq.GridQubit.square(4)


Maintenant nous pouvons voir comment créer et manipuler des portes logiques quantiques avec Cirq. La liste des portes ainsi que leur syntaxe est disponible [ici](https://cirq.readthedocs.io/en/stable/gates.html#Common-gates).
* [Portes à 1 qubit en entrée](https://cirq.readthedocs.io/en/stable/gates.html#Single-qubit-gates)
* [Créer sa propre porte](https://cirq.readthedocs.io/en/stable/gates.html#Advanced:-Creating-your-own-gates)

L'utilisation des portes est décrite ci-dessous.

In [15]:
import cirq
import matplotlib
# On peut créer une porte de la manière suivante
CNOT = cirq.CNOT

# Par elle même une porte ne fait rien, il faut lui fournir un Qubit pour obtenir un Qubit en sortie:
q0 = cirq.NamedQubit('q0')
QubitSortiePorte = cirq.X(q0)
print(QubitSortiePorte)

# On peut également obtenir leur matrice unitaire
Xgate = cirq.X
print(cirq.unitary(Xgate)) # Syntaxe: cirq.unitary(porte)

# Et même appliquer des opérateurs
XgateRoot = Xgate**0.5
print(cirq.unitary(XgateRoot))

# La porte de mesure est cirq.MeasurementGate( nb qubit, labels des qubits mesurés , ())(qubits mesurés)
print(cirq.MeasurementGate(1, 'q0',())(q0))

X(q0)
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]
cirq.MeasurementGate(1, 'q0', ())(q0)


Avec Cirq nous pouvons également créer des circuits, pour initialiser un circuit vide on utilise la ligne suivante: `circuit = cirq.Circuit()`
ensuite nous pouvons ajouter des portes comme illustré ci-dessous:

In [3]:
import cirq

# on commence par créer les qubits
q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(0, 1)
q2 = cirq.GridQubit(0, 2)

# On créé des portes en utilisant les qubits
CNOT01 = cirq.CNOT(q0, q1)
CNOT02 = cirq.CNOT(q0, q2)

# On initialise le circuit
circuit = cirq.Circuit()

# On ajoute les portes
circuit.append(CNOT01)
circuit.append(CNOT02)

print('Le circuit avec les portes')
print(circuit)



Le circuit avec les portes
(0, 0): ───@───@───
           │   │
(0, 1): ───X───┼───
               │
(0, 2): ───────X───


Cirq possède également les "Devices" ou appareil, qui sont des circuits préfabriqués ces appareils permettent égalemant de considérerd des limitations matérielles pour les exécutions sur processeur quantique et ainsi obtenir des résultats plus fidèles.

Enfin les différentes méthodes d'exécution, jusqu'ici nous avons vu comment créer un circuit, il existe deux méthodes pour l'exécuter, ces deux méthodes sont accessibles via `cirq.Simulator()` ce simulateur est intégré a Cirq et peut gérer jusqu'à 20 Qubits en même temps. Il peut être utilisé de deux manière différentes `simulate()` et `run()`. 
* `simulate()` permet de se familiariser avec cirq et d'obtenir directement la fonction d'onde
* `run()` permet de simuler la sortie d'un vrai processeur quantique et oblige donc à faire un échantillonnage.

Pour obtenir un vecteur de sortie:
`sortie = cirq.Simulator().simulate(circuit).final_state`


Ceci conclut l'introduction de Cirq, par manque de temps je n'ai pas écrit la partie sur les sommes de Pauli ou les autres fonctionnalités de Cirq, ainsi pour plus d'informations je vous conseille le [site officiel](https://cirq.readthedocs.io/en/stable/tutorial.html) (en anglais) de cirq.


## 2. Introduction de Tensorflow Quantum et exemple d'application

TFQ est un framework créé par Google pour faciliter le QML, il permet de travailler avec des données quantiques, ainsi que la construction de réseaux de neurones quantiques et hybrides classiques-quantiques.  


TFQ utilise des circuits de Cirq et des sommes de Pauli pour simuler les procédés quantiques, et utilise ces composants pour mettre en place des QNN.

TFQ va utiliser des tenseurs généré par la fonction `tfq.convert_to_tensor` qui va transformer des circuits Cirq ( ou des sommes de Pauli créées avec Cirq ) en tenseurs, cela permet de créer nos couches de neurones, et d'interfacer avec Tensorflow.

TFQ utilise donc des éléments quantiques créés par Cirq et joue le rôle d'interface avec Tensorflow. Il reste alors une question, comment implémenter des Qubits dans un réseau de neurones classiques ? 

Pour cela je propose de suivre l'exemple du [tutoriel sur le site de TFQ](https://www.tensorflow.org/quantum/tutorials/mnist), le cas de la reconnaissance d'image en utilisant la base de données MNIST étant un cas classique pour la formation en machine learning.

Pour créer un réseau de neurones utilisant les qubits, un QNN, il faut commencer par trouver comment nous allons les utiliser, ici les neurones du QNN vont être des portes logiques quantiques, et chaque pixel de l'image en entrée va être représentée par un qubit. Il faut donc adapter les couches de portes pour obtenir un résulat satisfaisant en sortie, pour faciliter la tache et réduire la charge de calculs, le simulateur intégré à Cirq étant limité en puissance, la taille du dataset, le nombre de label et la taille de chaque image est réduite, actuellement il semble peu probable que cet exemple puisse battre un réseau de neurones classique avec le dataset complet en considérant la précision en fonction du temps d'exécution.


## 3. Conclusion
 
Les réseaux de neurones sont actuellement en plein essor, et le principe utilisé qui consiste à se rapprocher suffisamment d'un minimum de la fonction d'erreur est pour nous, à même d'être grandement accéléré grâce aux processeurs quantiques, ainsi l'introduction de moyens permettant d'impliquer l'informatique quantique dans le développement de réseaux de neurones dès maintenant nous semble important.


## 4. Bibliographie et Sources

* [site officiel de Cirq](https://cirq.readthedocs.io/en/stable/)
* [site officiel de Tensorflow, section TFQ](https://www.tensorflow.org/quantum/overview)
* [tutoriel MNIST](https://www.tensorflow.org/quantum/tutorials/mnist)
